In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
import pandas as pd

# kaggle competitions download -c crowdflower-weather-twitter

In [ ]:
# Relevant links
# https://jax.readthedocs.io/en/latest/notebooks/quickstart.html
# https://stackoverflow.com/questions/63388668/valueerror-input-contains-nan-infinity-or-a-value-too-large-for-dtypefloat64
# https://www.kaggle.com/code/mohitsital/0-16372-predict-the-weather
# 

In [ ]:
# Make sure to upload the relevant kaggle files: train.csv, test.csv and sampleSubmission.csv
from google.colab import files
files.upload()

{}

In [ ]:
# read files and print shape
train, test, sub = pd.read_csv("train.csv"), pd.read_csv("test.csv") , pd.read_csv("sampleSubmission.csv")

print("Train",train.shape, "|", "Test",test.shape)

Train (77946, 28) | Test (42157, 4)


In [ ]:
# obtain target columns
targetColumns = train.columns.difference(test.columns)
print("Target Columns:")
print(targetColumns)

Target Columns:
Index(['k1', 'k10', 'k11', 'k12', 'k13', 'k14', 'k15', 'k2', 'k3', 'k4', 'k5',
       'k6', 'k7', 'k8', 'k9', 's1', 's2', 's3', 's4', 's5', 'w1', 'w2', 'w3',
       'w4'],
      dtype='object')


In [ ]:
# save said columns
target = train[targetColumns]
train.drop(targetColumns , axis = 1 , inplace = True)
print(pd.concat([train , test] , axis = 0 , ignore_index = True))

            id                                              tweet  \
0            1                Jazz for a Rainy Afternoon:  {link}   
1            2                   RT: @mention: I love rainy days.   
2            3  Good Morning Chicago! Time to kick the Windy C...   
3            6  Preach lol! :) RT @mention: #alliwantis this t...   
4            9                     @mention good morning sunshine   
...        ...                                                ...   
120098  120094  I kno the damn weather ppl betta stop interrup...   
120099  120096           Fuckkkk. Fucking snow. Get off my carrr.   
120100  120099  Good thing the weather alert app has a setting...   
120101  120101  Tornado record set: 292 in April and counting ...   
120102  120102     Track meet 2morro hope its nice ouTside #sunny   

               state              location  
0           oklahoma              Oklahoma  
1            florida  Miami-Ft. Lauderdale  
2              idaho                

In [ ]:
# flag missing location
for data in (train , test):
  missingLocationIndex = data.index[data.location.isnull()].tolist()
  missingLocationDictionary = {}
  for i in range(len(data)):
    if i in missingLocationIndex:
      missingLocationDictionary[i] = 1
    else:
      missingLocationDictionary[i] = 0
  data["missing_loc"] = data.index.map(missingLocationDictionary)

In [ ]:
# flag missing state
for data in (train , test):
  missingStateIDX = data.index[data.state.isnull()].tolist()
  missingStateDict = {}
  for i in range(len(data)):
    for i in missingStateIDX:
      missingStateDict[i] = 1
    else:
      missingStateDict[i] = 0
  data["missing_state"] = data.index.map(missingStateDict)

In [ ]:
# combining text of cols: location,state,tweet
for data in (train , test):
  data["location"] = data["location"].replace(jnp.nan , "" , regex = True)
  data["state"] = data["state"].replace(jnp.nan , "" , regex=True)
  data["full_text"] = data["tweet"] + " " + data["state"] + " " + data["location"]

In [ ]:
# replace abbvs with names
states = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming","NYC":"New York"}

# clean content
import re
def CleanPunctuation(s):
  res = re.sub(r'[?|!|\'|"|#]',r'' , s)
  res = re.sub(r'[.|,|)|(|)|\|/||:]',r' ' , res)      
  return res

def acronym(s, dct):
  return " ".join([dct.get(i , i) for i in s.split()])

for data in (train , test):
  dct = {}
  for i , row in data.iterrows():
    dct[i] = acronym(CleanPunctuation(row["full_text"]) , states)
  data["full_txt_2"] = data.index.map(dct)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Possible set union at position 14
  


In [ ]:
# mark features
for data in (train , test):
  data["num_word"] = data["tweet"].apply(lambda x : len(x.split()))
  data["num_char"] = data["tweet"].apply(lambda x : len(x))
  data["avg_word_length"]= data["num_char"] + 1 / data["num_word"]
print("done")

done


In [ ]:
# chklink
for data in (train , test):
  linkword = "{link)"
  linkDictionary = {}
  for i in range(len(data)):
    text = data.loc[i , "full_text"]
    if linkword in text.split():
      linkDictionary[i] = 1
    else:
      linkDictionary[i] = 0
  data["link_word"] = data.index.map(linkDictionary)

# chkmentions
for data in (train , test):
  mention = "@mention"
  mentionDictionary = {}
  for i in range(len(data)):
    text = data.loc[i , "full_text"]
    if mention in CleanPunctuation(text).split():
      mentionDictionary[i] = 1
    else:
      mentionDictionary[i] = 0
  data["mention_word"] = data.index.map(mentionDictionary)
print("done")

done


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

import re
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.corpus import stopwords

stopwords = set(stopwords.words("english"))
from wordcloud import WordCloud , STOPWORDS , ImageColorGenerator

def CleanHTML (s):
  res = re.sub(r'http\S+' , r'' , s)
  return res

def CleanPunctuation(s):
  res = re.sub(r'[?|!|\'|"|#]' , r'' , s)
  res = re.sub(r'[.|,|)|(|)|\|/|:]' , r' ' , res)      
  return res

for data in (train , test):
  s1=' '
  res=[]
  s2=''
  for sent in data["full_text"]:
    filter_sent = []
    remHTML = CleanHTML(sent)
    remPUNC = CleanPunctuation (remHTML)
    for w in remPUNC.split():
      if ((w.isalpha())):
        if (w.lower() not in stopwords):
          s2=(ps.stem(w.lower())).encode("utf8")
          s2=(w.lower()).encode("utf8")
          filter_sent.append(s2)
        else:
          continue
      else:
        continue
    s1 = b" ".join(filter_sent)
    res.append(s1)
  data["clean_text"] = np.array(res)

In [ ]:
# vectorize
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder

vectorizer = TfidfVectorizer(encoding="utf-8" , ngram_range=(1 , 3) , max_df = 0.75 , min_df = 5)

trainVector = vectorizer.fit_transform(train.clean_text)
testVector = vectorizer.transform(test.clean_text)

In [ ]:
# TruncatedSVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 2, algorithm = "randomized" , n_iter = 5 , random_state = None , tol = 0.0)
trainSVD = svd.fit_transform(trainVector)
testSVD = svd.transform(testVector)

In [ ]:
col = ["svd1" , "svd2"]
SVDTrain = pd.DataFrame(trainSVD , columns=col)
SVDTest = pd.DataFrame(testSVD , columns=col)

In [ ]:
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
vectorizer2 = TfidfVectorizer(encoding="utf-8" , ngram_range = (1 , 3) , max_df = 0.75 , min_df = 5 , max_features = 50000)

X = sp.sparse.hstack((vectorizer2.fit_transform(train.clean_text),normalize(train[["num_word" , "num_char" , "avg_word_length"]].values) , train[["missing_loc" , "missing_state" , "link_word" , "mention_word"]] , SVDTrain [["svd1" , "svd2"]]) , format = "csr")

X_columns = vectorizer.get_feature_names() + train[["num_word" , "num_char" , "avg_word_length" , "missing_loc" , "missing_state" , "link_word" , "mention_word"]].columns.tolist() + SVDTrain [["svd1" , "svd2"]].columns.tolist()
print(X.shape)

testSP = sp.sparse.hstack((vectorizer2.transform(test.clean_text) , normalize(test[["num_word" , "num_char" , "avg_word_length"]].values) , test[["missing_loc" , "missing_state" , "link_word" , "mention_word"]] , SVDTest [["svd1" , "svd2"]]) , format = "csr")

testColumns = vectorizer.get_feature_names() + test[["num_word" , "num_char" , "avg_word_length" , "missing_loc" , "missing_state" , "link_word" , "mention_word"]].columns.tolist() + SVDTest [["svd1" , "svd2"]].columns.tolist()
print(testSP.shape)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(77946, 32439)
(42157, 32439)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# splitting for cv
from sklearn.model_selection import train_test_split
x, x_test , y , y_test = train_test_split(X , target , test_size = 0.2 , train_size = 0.8 , random_state = 0)

In [ ]:
# running single model for each target columns
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.svm import LinearSVR

# Fit regression model




cols = target.columns
params = {"n_estimators" : 500 , "max_depth" : 3 , "min_samples_split": 2 , "learning_rate" : 0.01 , "loss" : "ls"}
predictionDictionary = {}
clf = Ridge(random_state = 25)
sum = 0
#print(type(clf))
for col in cols:
    print("start" , col)
    clf.fit(x , y[col])
    rmse = sqrt(mean_squared_error(y_test[col] , clf.predict(x_test)))
    print("RMSE: %.4f" % rmse)
    predictionDictionary[col] = rmse
    sum += rmse
averageRMSE = sum/24

start k1
RMSE: 0.0691
start k10
RMSE: 0.1243
start k11
RMSE: 0.0856
start k12
RMSE: 0.1363
start k13
RMSE: 0.1512
start k14
RMSE: 0.0504
start k15
RMSE: 0.0842
start k2
RMSE: 0.1420
start k3
RMSE: 0.0471
start k4
RMSE: 0.1570
start k5
RMSE: 0.0697
start k6
RMSE: 0.0216
start k7
RMSE: 0.2419
start k8
RMSE: 0.0324
start k9
RMSE: 0.1461
start s1
RMSE: 0.1145
start s2
RMSE: 0.2445
start s3
RMSE: 0.2177
start s4
RMSE: 0.2300
start s5
RMSE: 0.2085
start w1
RMSE: 0.2845
start w2
RMSE: 0.1761
start w3
RMSE: 0.2320
start w4
RMSE: 0.1546


In [ ]:
# how close we did 
print("Average RMSE: %.4f" % rmse)

Average RMSE: 0.1546
